In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/diabetes.csv', dtype=float)

In [7]:
X = df.drop('Outcome', axis=1).values
y = df['Outcome'].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
class DiabetesDataset(Dataset):
  def __init__(self, features, labels):
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.float32)

  def __len__(self):
    return len(self.features)
  
  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx]

In [10]:
train_dataset = DiabetesDataset(X_train, y_train)
test_dataset = DiabetesDataset(X_test, y_test)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [12]:
class DiabetesClassifier(nn.Module):
  def __init__(self):
    super(DiabetesClassifier, self).__init__()
    self.fc1 = nn.Linear(8, 64)
    self.fc2 = nn.Linear(64, 32)
    self.fc3 = nn.Linear(32, 1)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x

In [13]:
model = DiabetesClassifier()

In [14]:
# loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
epochs = 10
for epoch in range(epochs):
  model.train()
  running_loss = 0.0
  
  for inputs, labels in train_loader:
    optimizer.zero_grad() # zero the parameter gradients
    outputs = model(inputs) # forward pass
    loss = criterion(outputs, labels.unsqueeze(1))
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  
  print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss/len(train_loader)}')

Epoch 1/10, Loss: 0.41594360768795013
Epoch 2/10, Loss: 0.39277745932340624
Epoch 3/10, Loss: 0.3872130498290062
Epoch 4/10, Loss: 0.3976041205227375
Epoch 5/10, Loss: 0.4144726887345314
Epoch 6/10, Loss: 0.4000185117125511
Epoch 7/10, Loss: 0.39417013674974444
Epoch 8/10, Loss: 0.39313490614295005
Epoch 9/10, Loss: 0.39135552793741224
Epoch 10/10, Loss: 0.39789171814918517


In [47]:
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
  for inputs, labels in test_loader:
    outputs = model(inputs)
    predicted_labels = (outputs >= 0.5).float()
    total_correct += (predicted_labels == labels.unsqueeze(1)).sum()
    total_samples += labels.size(0)
  
accuracy = total_correct / total_samples
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.70
